In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq

# Problem 1

In [37]:
#Will use Python's objects just to store variables, without defining any methods.
#(Alternatively, storing a collection of named attributes could be done 
#without introducing classes/objects, for instance by using Python's dictionary structures.
#Decided to use classes/objects here, in order to facilitate possible future usage of OOP, if desired.)

class Contract:
    pass

In [38]:
hw1contract=Contract()
hw1contract.K = 95
hw1contract.T = 0.25
hw1contract.H = 114
hw1contract.observationinterval = 0.02

In [39]:
class Dynamics:
    pass

In [40]:
hw1dynamics=Dynamics()
hw1dynamics.S0 = 100
hw1dynamics.sigma = 0.4
hw1dynamics.r = 0

In [41]:
class Tree:
    pass

In [42]:
hw1tree=Tree()
hw1tree.N=20000

In [77]:
def barrier_trinom_pricer(dynamics, contract, tree):
    
    S0, sigma, r = dynamics.S0, dynamics.sigma, dynamics.r
    K, T, H, interval  = contract.K, contract.T, contract.H, contract.observationinterval
    N = tree.N

    deltat =  T/N#Fill this in with a scalar.
    if H!=0:
        j_star = round((np.log(H)-np.log(S0))/(sigma*np.sqrt(3*deltat))-0.5)  #Fill this in with a scalar.
        deltax = (np.log(H)-np.log(S0))/(j_star+0.5)
    else:
        deltax = sigma * np.sqrt(3*deltat) ## use this to verify without barrier, or else log(H) will make nonsense
        
    S=S0*np.exp(np.linspace(N, -N, num=2*N+1, endpoint=True)*deltax)  
    #Here I decided to make the SMALLER indexes in this array correspond to HIGHER S
    if abs(interval/deltat-round(interval/deltat)) > 1e-12:
        raise ValueError("This value of N fails to place the observation dates in the tree.")

    nu = r-sigma**2/2       # complete this 
    Pu = 0.5*((sigma**2*deltat+(nu*deltat)**2)/(deltax**2)+(nu*deltat/deltax))       # complete this
    Pd = 0.5*((sigma**2*deltat+(nu*deltat)**2)/(deltax**2)-(nu*deltat/deltax))       # complete this
    Pm = 1-(sigma**2*deltat+(nu*deltat)**2)/(deltax**2)       # complete this
        
    optionprice = np.maximum(K-S,0)   #an array of time-T option prices.
    #Next, induct backwards to time 0, updating the optionprice array 
    #Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]
    observations = [interval*(1+i) for i in range(int(T/interval))]
    for t in np.linspace(N-1, 0, num=N, endpoint=True)*deltat:
        # insert lines of code here if needed
        up_state = (optionprice*Pu)[:-2]
        mid_state = (optionprice*Pm)[1:-1]
        down_state = (optionprice*Pd)[2:]
        S = S[1:-1]
        optionprice = np.exp(-r*deltat)*(up_state+mid_state+down_state)   #complete this

        for check in observations:
            if abs(check-t)<1e-10:
                if H>0:
                    optionprice = np.where(S<H,optionprice,0)  ## override for barrier case. checked at evert 0.02 interval
                    print(optionprice)
    return optionprice[0]   
    #The [0] is assuming that we are shrinking the optionprice array in each iteration of the loop,
    #until finally there is only 1 element in the array.
    #If instead you are keeping unchanged the size of the optionprice array in each iteration, 
    #then you need to change the [0] to a different index.



In [78]:
knock_out_P0 = barrier_trinom_pricer(hw1dynamics, hw1contract, hw1tree)
knock_out_P0

5.519478970919443

# Problem 1 - Verify

In [65]:
def BScallPrice(sigma,S,rGrow,r,K,T):
    F=S*np.exp(rGrow*T)
    sd = sigma*np.sqrt(T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))

In [66]:
def BSputPrice(sigma,S,rGrow,r,K,T):
    call = BScallPrice(sigma,S,rGrow,r,K,T)
    F = S - K*np.exp(-rGrow*T)
    return call - F

In [67]:
BSput = BSputPrice(0.4,100,0,0,95,0.25)
print("BS-put has the price as {:4f}".format(BSput))

BS-put has the price as 5.519541


In [68]:
hw1contract.H = 0  ## assuem no barrier
Treeput = barrier_trinom_pricer(hw1dynamics, hw1contract, hw1tree)
print("Trenomial Tree-put has the price at time-0 as {:4f}".format(Treeput))

Trenomial Tree-put has the price at time-0 as 5.519479


# Problem 1 - b

since {1 knock-out-put, 1 knock_in_put} can perfectly replicate a vanilla put with the same parameters, we can caluclate the knoco-in-put by no-arbitrage rule at T0.

In [79]:
knock_in_P0=BSput-knock_out_P0
knock_in_P0

6.209275753210619e-05

# Problem 1 - c1

If monitored continuously, more points in up scenarios could become zero, which decrease the price

# Problem 2 - a

In [84]:
def IVofCall(C,S,rGrow,r,K,T):
    F=S*np.exp(rGrow*T)
    lowerbound = np.max([0,(F-K)*np.exp(-r*T)])
    if C<lowerbound:
        return np.nan
    if C==lowerbound:
        return 0
    if C>=F*np.exp(-r*T):
        return np.nan 
    hi=0.2
    while BScallPrice(hi,S,rGrow,r,K,T)>C:
        hi=hi/2
    while BScallPrice(hi,S,rGrow,r,K,T)<C:
        hi=hi*2
    lo=hi/2
    # We have calculated "lo" and "hi" which bound the implied volatility from below and above. 
    # In other words, the implied volatility is somewhere in the interval [lo,hi].
    # Then, to calculate the implied volatility within that interval, 
    # for purposes of this homework, you may either (A) write your own bisection algorithm, 
    # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
    # You will need to provide lo and hi to those solvers.
    # There are other solvers that do not require you to bound the solution 
    # from below and above (for instance, scipy.optimize.fsolve is a useful solver).  
    # However, if you are able to bound the solution (of a single-variable problem), 
    # then bisection or Brent will be more reliable.
    
    def f(x):
        return BScallPrice(x,S,rGrow,r,K,T)-C
    
    impliedVolatility = brentq(f,a=0.01,b=0.99)     # you fill this in, using bisect or brentq imported from scipy.optimize,
                             # or by writing your own bisection algorithm.
    return impliedVolatility

In [88]:
vol1 = IVofCall(11.25,100,0,0,100,0.5)
vol1

0.40013278092106896

In [90]:
vol2=IVofCall(12,100,0,0,100,1)
vol2

0.30193843099358947

# Problem 2 - b

In [91]:
vol3 = 0.5*(vol1+vol2)
call3 = BScallPrice(vol3,100,0,0,100,0.75)
call3

12.0815332862499

# Problem 2 - b

In [ ]:
call3 should have a lower price due to the convexity in volatility -- its price should be somewhere between 11.25 and 12.
Thus we can (-C0.7)